In [1]:
from os import listdir
from os.path import isfile, isdir
import pandas as pd

In [2]:
SOURCE_PATH = './Schema Mediato/Datasets Original/'
CLEANED_DATASETS_PATH = './Cleaned Datasets/'
COLUMNS_TO_DROP = ['id', 'Unnamed: 0']

In [23]:
def clean_dataset(dataset):
    for col in COLUMNS_TO_DROP:
        if col in dataset.columns:
            dataset.drop(col, axis=1, inplace=True)

    dataset.dropna(axis=0, how='all', inplace=True)
    dataset.fillna('', inplace=True)

    for i in dataset:
        dataset[i] = [', '.join(map(str, l)) if isinstance(l, list) else l for l in dataset[i]]

    for i in dataset:
        if dataset[i].dtype == object:
            dataset[i] = dataset[i].str.lower()
            dataset[i] = dataset[i].str.replace('mr|ms|[^\w\s]', '', regex=True)
            dataset[i] = dataset[i].str.replace('none', '', regex=True)
            dataset[i] = dataset[i].str.replace('[0-9]\sb$|[0-9]\sbillion$', '000000000', regex=True)
            dataset[i] = dataset[i].str.replace('[0-9]\st$|[0-9]\strillion$', '000000000000', regex=True)
            dataset[i] = dataset[i].str.strip()

    return dataset

for elem in listdir(SOURCE_PATH):
    dir_path = SOURCE_PATH + elem
    if isdir(dir_path):
        for f in listdir(dir_path):
            file_path = dir_path + '/' + f
            csv_file_name = file_path[file_path.rfind('/') + 1:]
            json_file_path = CLEANED_DATASETS_PATH
            df = None

            if isfile(file_path):
                if file_path.endswith('.csv'):
                    json_file_name = csv_file_name.replace('.csv', '.jsonl')
                    json_file_path = json_file_path + json_file_name

                    d = {
                        'csv_file_name': csv_file_name,
                        'json_file_name': json_file_name,
                    }

                    try:
                        df = pd.read_csv(file_path, encoding='utf-8')
                        d['encoding'] = 'utf-8'
                        d['outcome'] = 'ok'
                    except UnicodeDecodeError:
                        df = pd.read_csv(file_path, encoding='unicode_escape')
                        d['encoding'] = 'unicode_escape'
                        d['outcome'] = 'ok'
                    except:
                        d['outcome'] = 'ko'

                    print(d)
                elif file_path.endswith('.json'):
                    json_file_name = csv_file_name.replace('.json', '.jsonl')
                    json_file_path = json_file_path + json_file_name
                    df = pd.read_json(file_path, encoding='utf-8')
                elif file_path.endswith('jsonl'):
                    json_file_path = file_path
                    df = pd.read_json(file_path, encoding='utf-8', lines=True)

                if df is not None:
                    clean_dataset(df)
                    df.to_json(json_file_path, orient='records', lines=True, force_ascii=False)

{'csv_file_name': 'value.today_silvestri.csv', 'json_file_name': 'value.today_silvestri.jsonl', 'encoding': 'utf-8', 'outcome': 'ok'}
{'csv_file_name': 'ft_fr.csv', 'json_file_name': 'ft_fr.jsonl', 'encoding': 'utf-8', 'outcome': 'ok'}
{'csv_file_name': 'ft_DDD.csv', 'json_file_name': 'ft_DDD.jsonl', 'encoding': 'utf-8', 'outcome': 'ok'}
{'csv_file_name': 'ft_silvestri.csv', 'json_file_name': 'ft_silvestri.jsonl', 'encoding': 'utf-8', 'outcome': 'ok'}
{'csv_file_name': 'Disfold_MarScoToc.csv', 'json_file_name': 'Disfold_MarScoToc.jsonl', 'encoding': 'unicode_escape', 'outcome': 'ok'}
{'csv_file_name': 'disfold_fr.csv', 'json_file_name': 'disfold_fr.jsonl', 'encoding': 'utf-8', 'outcome': 'ok'}
{'csv_file_name': 'disfold_silvestri.csv', 'json_file_name': 'disfold_silvestri.jsonl', 'encoding': 'utf-8', 'outcome': 'ok'}
{'csv_file_name': 'CompaniesMarketCap_MarScoToc.csv', 'json_file_name': 'CompaniesMarketCap_MarScoToc.jsonl', 'encoding': 'unicode_escape', 'outcome': 'ok'}
{'csv_file_nam